In [76]:
"""
1. 이미지를 resnet에 넣고 npz 파일로 저장
datastructure
{
  'front_view': np.ndarray of shape [total_length, T, 512],
  'top_view':   np.ndarray of shape [total_length, T, 512],
  'wrist_view': np.ndarray of shape [total_length, T, 512],
}

"""
import os
import re
import torch
import numpy as np
import pickle
from PIL import Image
from tqdm import tqdm
from torchvision import models, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

# ResNet18 feature extractor
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Identity()  # remove final fc
resnet18 = resnet18.to(device)
resnet18.eval()
#print("Weight hash:", hash(str(resnet18.state_dict())))
# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

def extract_features(image_dir):
    def extract_index(filename):
        match = re.search(r'front_view_(\d+)\.png', filename)
        return int(match.group(1)) if match else -1

    image_files = sorted(
        [f for f in os.listdir(image_dir) if f.endswith(".png")],
        key=extract_index
    )

    features = []

    for fname in image_files:
        #print(fname)
        img = Image.open(os.path.join(image_dir, fname)).convert('RGB')
        img_tensor = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            feat = resnet18(img_tensor)
        features.append(feat.cpu().numpy()[0])

    return np.stack(features, axis=0)  # shape: [T, 512]

def get_episode_number(ep_dir_name):
    match = re.search(r"success_episode(\d+)_steps(\d+)", ep_dir_name)
    return int(match.group(1)) if match else -1

def process_all_episodes(root_dir, output_npz):
    front_list, top_list, wrist_list = [], [], []

    episode_dirs = sorted(
        [d for d in os.listdir(root_dir) if d.startswith("success_episode")],
        key=get_episode_number
    )
   
    for i, ep_dir in enumerate(tqdm(episode_dirs, desc="Processing episodes")):
        ep_path = os.path.join(root_dir, ep_dir)
        print(f"[{i+1}/{len(episode_dirs)}] Processing: {ep_dir}")

        try:
            front_feat = extract_features(os.path.join(ep_path, "front_view"))
            top_feat   = extract_features(os.path.join(ep_path, "top_view"))
            wrist_feat = extract_features(os.path.join(ep_path, "wrist_view"))
        except Exception as e:
            print(f"❌ Error in {ep_dir}: {e}")
            continue

        # Optionally check consistency of T
        T_f, T_t, T_w = len(front_feat), len(top_feat), len(wrist_feat)
        if T_f != T_t or T_t != T_w:
            print(f"⚠️ Skipping {ep_dir} due to mismatched frame counts: front={T_f}, top={T_t}, wrist={T_w}")
            continue

        front_list.append(front_feat)
        top_list.append(top_feat)
        wrist_list.append(wrist_feat)

    # Save as object arrays (list of np.ndarray)
    np.savez_compressed(
        output_npz,
        front_view=np.array(front_list, dtype=object),
        top_view=np.array(top_list, dtype=object),
        wrist_view=np.array(wrist_list, dtype=object)
    )
    print(f"✅ Saved to {output_npz}")


# 실행
process_all_episodes("/AILAB-summer-school-2025/success_data_raw", "dataset/image_features(resnet18).npz")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Processing episodes:   0%|          | 0/505 [00:00<?, ?it/s]

[1/505] Processing: success_episode1_steps320


Processing episodes:   0%|          | 1/505 [00:00<05:16,  1.59it/s]

[2/505] Processing: success_episode2_steps316


Processing episodes:   0%|          | 2/505 [00:01<04:50,  1.73it/s]

...

Processing episodes:   2%|▏         | 9/505 [00:04<04:05,  2.02it/s]

[10/505] Processing: success_episode10_steps319


Processing episodes:   2%|▏         | 10/505 [00:04<04:06,  2.01it/s]

[11/505] Processing: success_episode11_steps353


KeyboardInterrupt: 

In [ ]:

"""
2. npz 파일 shape확인
"""
import numpy as np

def inspect_image_features(npz_path):
    data = np.load(npz_path, allow_pickle=True)

    print(f"📁 Loaded file: {npz_path}")
    print(f"🔑 Keys in file: {list(data.keys())}\n")

    for key in data.files:
        array = data[key]
        print(f"▶️ Key: '{key}'")
        print(f"   - Shape: {array.shape}")
        print(f"   - Dtype: {array.dtype}")
        print(f"   - Example values (first 1 row):\n{len(array[0][0])}")
        print()

# 사용 예시
npz_file_path = "/AILAB-summer-school-2025/scripts/make_dataset/image_features(resnet18).npz"
inspect_image_features(npz_file_path)


📁 Loaded file: /AILAB-summer-school-2025/scripts/make_dataset/image_features(resnet18).npz
🔑 Keys in file: ['front_view', 'top_view', 'wrist_view']

▶️ Key: 'front_view'
   - Shape: (6,)
   - Dtype: object
   - Example values (first 1 row):
1.7124043703079224

▶️ Key: 'top_view'
   - Shape: (6,)
   - Dtype: object
   - Example values (first 1 row):
0.6083284616470337

▶️ Key: 'wrist_view'
   - Shape: (6,)
   - Dtype: object
   - Example values (first 1 row):
2.733461856842041



In [31]:
import os
import re
import numpy as np
import torch
from PIL import Image
from torchvision import models, transforms

# ✅ ResNet18 정의 (512차원 출력)
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = torch.nn.Identity()
resnet18.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet18 = resnet18.to(device)
print("Weight hash:", hash(str(resnet18.state_dict())))

# ✅ 이미지 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

# ✅ 설정
npz_path = "/AILAB-summer-school-2025/scripts/make_dataset/image_features(resnet18).npz"
success_data_raw = "/AILAB-summer-school-2025/success_data_raw"
episode_idx = 25  # 검증할 에피소드 인덱스
stride = 5       # 저장된 프레임 간격
view_names = ["front_view", "top_view", "wrist_view"]

# ✅ .npz 로딩
data = np.load(npz_path, allow_pickle=True)

# ✅ 에피소드 디렉토리 정렬
episode_dirs = sorted(
    [d for d in os.listdir(success_data_raw) if d.startswith("success_episode")],
    key=lambda name: int(re.search(r"success_episode(\d+)", name).group(1))
)
episode_name = episode_dirs[episode_idx]
episode_path = os.path.join(success_data_raw, episode_name)

print(f"🔍 Checking episode: {episode_name}, stride={stride}")

# ✅ View 별 점검 루프
for view in view_names:
    print(f"\n🔎 Checking view: {view}")
    ep_feat_seq = data[view][episode_idx]  # shape [T, 512]
    T = ep_feat_seq.shape[0]
    view_dir = os.path.join(episode_path, view)

    num_fail = 0
    for i in range(T):
        frame_number = i * stride
        filename = f"{view}_{frame_number}.png"
        filepath = os.path.join(view_dir, filename)

        if not os.path.exists(filepath):
            print(f"❌ Missing file: {filepath}")
            num_fail += 1
            continue

        image = Image.open(filepath).convert("RGB")
        img_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            feat_from_resnet = resnet18(img_tensor).cpu().numpy().squeeze()

        feat_from_npz = ep_feat_seq[i]
        is_same = np.allclose(feat_from_npz, feat_from_resnet, atol=1e-6)

        if not is_same:
            diff = np.abs(feat_from_npz - feat_from_resnet)
            print(f"⚠️  Frame {i} (image {frame_number}.png): mismatch")
            print(f"   → Max diff: {np.max(diff):.6f}, Mean diff: {np.mean(diff):.6f}")
            num_fail += 1

    if num_fail == 0:
        print(f"✅ {view}: All features match.")
    else:
        print(f"❌ {view}: {num_fail}/{T} frames failed consistency check.")


Weight hash: -1720887647942330811
🔍 Checking episode: success_episode26_steps313, stride=5

🔎 Checking view: front_view
✅ front_view: All features match.

🔎 Checking view: top_view
✅ top_view: All features match.

🔎 Checking view: wrist_view
✅ wrist_view: All features match.


In [ ]:
"""
PCA 학습 및 적용 스크립트
- 중요한 점: 각 view (front_view, top_view, wrist_view) 마다 개별적으로 PCA를 학습하고 적용함
"""

import numpy as np
import os
import pickle
from sklearn.decomposition import PCA
from tqdm import tqdm

# === 설정 ===
input_path = "/AILAB-summer-school-2025/scripts/make_dataset/image_features(resnet18).npz"
output_path = "/AILAB-summer-school-2025/scripts/make_dataset/image_features(pca).npz"
pca_dir = "/AILAB-summer-school-2025/scripts/make_dataset"

views = ["front_view", "top_view", "wrist_view"]
compressed_dim = 64

# === 데이터 로드 ===
data = np.load(input_path, allow_pickle=True)
data = dict(data)

# === PCA 적용 결과 저장용 리스트 ===
compressed_data = {
    "front_view": [],
    "top_view": [],
    "wrist_view": []
}

# === View별 PCA 학습 및 적용 ===
for view in views:
    print(f"\n[INFO] Processing view: {view}")

    # 전체 feature 수집
    all_feats = []
    for epi in data[view]:
        for step_feat in epi:
            all_feats.append(step_feat)
    all_feats = np.stack(all_feats)  # shape: (N, 512)

    print(f"[INFO] View '{view}': Collected {all_feats.shape[0]} features")

    # PCA 학습
    pca = PCA(n_components=compressed_dim)
    pca.fit(all_feats)
    print(f"[INFO] PCA for {view}: Explained variance = {pca.explained_variance_ratio_.sum():.4f}")

    # PCA 저장
    pca_path = os.path.join(pca_dir, f"pca_{view}.pkl")
    with open(pca_path, "wb") as f:
        pickle.dump(pca, f)
    print(f"[INFO] Saved PCA model to {pca_path}")

    # PCA 변환 적용
    for epi in tqdm(data[view], desc=f"Compressing {view}"):
        compressed_epi = []
        for step_feat in epi:
            step_feat = step_feat.reshape(1, -1)  # shape: (1, 512)
            compressed_feat = pca.transform(step_feat).flatten()  # shape: (64,)
            compressed_epi.append(compressed_feat)
        compressed_data[view].append(np.stack(compressed_epi))  # shape: (T, 64)

# === npz로 저장 (dtype=object로 리스트 구조 유지) ===
np.savez_compressed(
    output_path,
    front_view=np.array(compressed_data["front_view"], dtype=object),
    top_view=np.array(compressed_data["top_view"], dtype=object),
    wrist_view=np.array(compressed_data["wrist_view"], dtype=object)
)

print(f"\n✅ Saved PCA-compressed features (64-dim) to {output_path}")



[INFO] Processing view: front_view
[INFO] View 'front_view': Collected 32858 features
[INFO] PCA for front_view: Explained variance = 0.9478
[INFO] Saved PCA model to /AILAB-summer-school-2025/scripts/make_dataset/pca_front_view.pkl


Compressing front_view: 100%|██████████| 505/505 [00:02<00:00, 222.96it/s]



[INFO] Processing view: top_view
[INFO] View 'top_view': Collected 32858 features
[INFO] PCA for top_view: Explained variance = 0.9659
[INFO] Saved PCA model to /AILAB-summer-school-2025/scripts/make_dataset/pca_top_view.pkl


Compressing top_view: 100%|██████████| 505/505 [00:01<00:00, 272.69it/s]



[INFO] Processing view: wrist_view
[INFO] View 'wrist_view': Collected 32858 features
[INFO] PCA for wrist_view: Explained variance = 0.9397
[INFO] Saved PCA model to /AILAB-summer-school-2025/scripts/make_dataset/pca_wrist_view.pkl


Compressing wrist_view: 100%|██████████| 505/505 [00:01<00:00, 286.77it/s]



✅ Saved PCA-compressed features (64-dim) to /AILAB-summer-school-2025/scripts/make_dataset/image_features_pca.npz


In [75]:

"""
4. pca 파일 shape확인
"""
import numpy as np

def inspect_image_features(npz_path):
    data = np.load(npz_path, allow_pickle=True)

    print(f"📁 Loaded file: {npz_path}")
    print(f"🔑 Keys in file: {list(data.keys())}\n")

    for key in data.files:
        array = data[key]
        print(f"▶️ Key: '{key}'")
        print(f"   - Shape: {array.shape}")
        print(f"   - Dtype: {array.dtype}")
        print(f"   - Example values (first 1 row):\n{len(array[:1])}")
        print()

# 사용 예시
npz_file_path = "/AILAB-summer-school-2025/scripts/make_dataset/image_features(pca).npz"
inspect_image_features(npz_file_path)


📁 Loaded file: /AILAB-summer-school-2025/scripts/make_dataset/image_features(pca).npz
🔑 Keys in file: ['front_view', 'top_view', 'wrist_view']

▶️ Key: 'front_view'
   - Shape: (505,)
   - Dtype: object
   - Example values (first 1 row):
1

▶️ Key: 'top_view'
   - Shape: (505,)
   - Dtype: object
   - Example values (first 1 row):
1

▶️ Key: 'wrist_view'
   - Shape: (505,)
   - Dtype: object
   - Example values (first 1 row):
1

